# Decision Trees

En este ejercicio, vamos a implementar un árbol de decisión desde 0, empleandolo para clasificar setas en venenosas o no venenosas, en función de sus características.


## 1 - Importar paquetes

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from utils import *

%matplotlib inline

## 2 -  Descripción del problema

Supongamos que está creando una empresa que cultiva y vende setas silvestres. 
- Necesitamos saber si una determinada seta es comestible o venenosa basándose en sus atributos físicos.


## 3 - Dataset

Cargamos el dataset, que tiene el siguiente aspecto

|                                                     | Cap Color | Stalk Shape | Solitary | Edible |
|:---------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    Yes   |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |    Red    |  Enlarging  |    No    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    No    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |


- Disponemos de 10 especies. Cada seta es un ejemplo para el que tenemos:
    - Tres características (features): Xs
        - Color del "sombrero" (`Marron` or `Red`),
        - Forma del pie (`Tapering (as in \/)` or `Enlarging (as in /\)`), and
        - Solitaria (`Yes` or `No`)
    - Etiqueta: y
        - Comestible (`1` indicando si `0` indicando no (es venenosa))


### 3.1 One hot encoding
Convertimos los valores en 1s o 0s para que el algoritmo pueda trabajar con ellos.

|                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |


- `X_train` contiene las características de cada ejemplo (especie de seta) 
    - Color marrón (`1` indica "Marrón" y `0` indica "Rojo")
    - Forma del pie (`1` indica "Forma de pie cónico" y `0` indica "Forma de pie ensanchado")
    - Solitaria  (`1` indica "Sí" y `0` indica "No")


- `y_train` indica si la seta es comestible o no
    - `y = 1` indica comestible
    - `y = 0` indica venenosa

In [3]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [4]:
print("Primeros elementos de X_train:\n", X_train[:5])
print("Tipo de X_train:",type(X_train))
print('Forma de X_train is:', X_train.shape)

print("\n" + "-"*100 + "\n")

print("Primeros elementos de y_train:", y_train[:5])
print("Tipo de y_train:",type(y_train))
print("Forma de y_train:", y_train.shape)

print("\n" + "-"*100 + "\n")

print('Número de ejemplos de entrenamiento (m):', len(X_train))

Primeros elementos de X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]
Tipo de X_train: <class 'numpy.ndarray'>
Forma de X_train is: (10, 3)

----------------------------------------------------------------------------------------------------

Primeros elementos de y_train: [1 1 0 0 1]
Tipo de y_train: <class 'numpy.ndarray'>
Forma de y_train: (10,)

----------------------------------------------------------------------------------------------------

Número de ejemplos de entrenamiento (m): 10


## 4 - Decision Tree


Los pasos para construir un árbol de decisión son los siguientes:
- Comenzar con todos los ejemplos en el nodo raíz
- Calcular la ganancia de información para dividir en todas las características posibles y elegir la que tenga la mayor ganancia de información
- Dividir el conjunto de datos según la característica seleccionada y crear ramas izquierda y derecha del árbol
- Repetir el proceso de división hasta que se cumpla el criterio de parada

Vamos a prepararlas siguientes funciones para construir el árbol de decisión:
- Calcular la entropía en un nodo
- Dividir el conjunto de datos en un nodo en ramas izquierda y derecha basadas en una característica dada
- Calcular la ganancia de información de dividir en una característica dada
- Elegir la característica que maximice la ganancia de información

Repetiremos el proceso de división hasta que se cumpla el criterio de parada. 
En este ejercicio, el criterio de parada es establecer una profundidad máxima de 2.
    

### 4.1  Calcular entropía

Preparamos una función para calcular la entrepía en un nodo, esto es, la impureza de los datos en el nodo.

La función toma un array (`y`) que indica si los ejemplos en ese nodo son comestibles (`1`) o venenosos (`0`).
El resultado es un valor que indica la impureza (proporción de ejemplos venenosos) en ese nodo.

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$

* Notas:
    * El logaritmo se calcula con base $2$
    * Para fines de implementación, $0\text{log}_2(0) = 0$. Es decir, si `p_1 = 0` o `p_1 = 1`, la entropía es `0`

In [5]:
def compute_entropy(y):
    """
    Computes the entropy for 
    
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
       
    Returns:
        entropy (float): Entropy at that node
        
    """
    entropy = 0.
    
    if len(y) != 0:   
        p1 = len(y[y==1]) / len(y)
        p0 = 1-p1
        
        if p1 != 0 and p1 != 1:
            entropy = -p1*np.log2(p1)-p0*np.log2(p0)   
    
    return entropy

Ejecutamos la función con todos los datos, root node:

In [6]:
# En los datos de entrada tenemos 5 setas comestibles y 5 venenosas, por lo que la entropía es 1.

print("Entropía root node: ", compute_entropy(y_train)) 

Entropía root node:  1.0


### 4.2 Dividir los datos por ramas izquierda y derecha

Escribiremos una función auxiliar llamada `split_dataset` que toma los datos en un nodo y una característica para dividir y los divide en ramas izquierda y derecha:
- La función toma los datos en un nodo, `X`, y una característica para dividir, `feature`.
- La función devuelve los índices de los datos en `X` que van a la rama izquierda y los índices de los datos en `X` que van a la rama derecha.
- Por ejemplo, en el nodo raíz, `node_indices = [0,1,2,3,4,5,6,7,8,9]`, y elegimos dividir en la característica `0`, que es si la seta tiene un sombrero marrón.
    - La salida de la función es entonces, `left_indices = [0,1,2,3,4,7,9]` (puntos de datos con sombrero marrón) y `right_indices = [5,6,8]` (puntos de datos sin sombrero marrón)
    
    
|       |                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|-------|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| 0     | <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| 1     | <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| 2     | <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| 3     | <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| 4     | <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| 5     | <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| 6     | <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| 7     | <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| 8     | <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| 9     | <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |


In [6]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """
    
    # You need to return the following variables correctly
    left_indices = []
    right_indices = []
    
    for i in node_indices:
        if X[:i+1, feature][i] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)
        
    return left_indices, right_indices

Probamos la función y visualizamos los resultados:

In [50]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# Visualize the split 
#generate_split_viz(root_indices, left_indices, right_indices, feature)

Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]


<img src="images/vis1.png" alt="drawing" width="1000"/>

### 4.3  Calcular ganancia de información

Ahora que tenemos una función para dividir los datos en ramas izquierda y derecha, podemos calcular la ganancia de información de dividir en una característica dada.gain from the split.

$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

Dónde: 
- $H(p_1^\text{node})$ es la entropía en el nodo antes de la división
- $H(p_1^\text{left})$ and $H(p_1^\text{right})$ son las entropías en las ramas izquierda y derecha después de la división
- $w^{\text{left}}$ and $w^{\text{right}}$ son las proporciones de ejemplos en las ramas izquierda y derecha, respectivamente


In [53]:
def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    entropy_node = compute_entropy(y_node)
    entropy_left = compute_entropy(y_left)
    entropy_right = compute_entropy(y_right)
    
    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)
    
    information_gain = entropy_node - ((w_left * entropy_left) + (w_right * entropy_right))
    
    return information_gain

Ejecutamos la función en el root node, probando cada una de las tres características:

In [54]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain resultado de dividir por color del sombrero (marrón): ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain resultado de dividir por la forma del tallo: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain resultado de dividir en función de si la seta se encuentra en solitario: ", info_gain2)


Information Gain resultado de dividir por color del sombrero (marrón):  0.034851554559677034
Information Gain resultado de dividir por la forma del tallo:  0.12451124978365313
Information Gain resultado de dividir en función de si la seta se encuentra en solitario:  0.2780719051126377


Dividir por "solitary" (feature = 2) en el nodo raíz da la máxima ganancia de información. Por lo tanto, es la mejor característica para comenzar.

### 4.4  Get best split
Por último, escribiremos una función para obtener la mejor característica para dividir en un nodo dado.
- La función toma los datos en un nodo, `X`, y devuelve el índice de la característica que da la máxima ganancia de información.


In [27]:
def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    max_info_gain = 0
    
    for feature in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature=feature)
        
        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = feature
   
    return best_feature

Ejecutamos la función:

In [55]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

Best feature to split on: 2


<a name="5"></a>
## 5 - Generar el árbol de decisión

Utilizamos las funciones anteriores para generar un árbol de decisión, estableciendo una profundidad máxima de 2.

In [56]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [1]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=3, current_depth=0)
#generate_tree_viz(root_indices, y_train, tree)

NameError: name 'build_tree_recursive' is not defined

<img src="images/vis2.png" alt="drawing" width="1000"/>

# Decision Trees: Sampling With Replacement

Vamos a repetir el proceso anterior, pero esta vez, en lugar de dividir en la característica que da la máxima ganancia de información, elegiremos una característica al azar de las tres características posibles.

In [31]:
# Function to generate bootstrap samples
def generate_bootstrap_samples(X_train, y_train, B):
    """
    Generate B bootstrapped samples from the training data
    
    Args:
        X_train (ndarray): Data matrix of shape(n_samples, n_features)
        y_train (array like): list or ndarray with n_samples containing the target variable
        B (int): Number of bootstrapped samples to generate
    
    Returns:
        X_bootstrap (ndarray): Data matrix of shape(B, n_samples, n_features)
        y_bootstrap (ndarray): Data matrix of shape(B, n_samples)
    """
    
    # You need to return the following variables correctly
    X_bootstrap = []
    y_bootstrap = []
    
    for i in range(B):
        indices = np.random.choice(range(len(X_train)), len(X_train), replace=True)
        X_bootstrap.append(X_train[indices])
        y_bootstrap.append(y_train[indices])
    
    return np.array(X_bootstrap), np.array(y_bootstrap)

In [62]:
# Num trees B = 3
B = 3

# Generate bootstrap samples
bootstrapped_X, bootstrapped_y = generate_bootstrap_samples(X_train, y_train, B)

print("Bootstrapped X shape: ", bootstrapped_X.shape)
print("Bootstrapped y shape: ", bootstrapped_y.shape)

for i in range(B):
    print("\n" + "-"*100 + "\n")
    print("Bootstrapped X for tree %d:\n" % i, bootstrapped_X[i])
    print("Bootstrapped y for tree %d:\n" % i, bootstrapped_y[i])


Bootstrapped X shape:  (3, 10, 3)
Bootstrapped y shape:  (3, 10)

----------------------------------------------------------------------------------------------------

Bootstrapped X for tree 0:
 [[1 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [1 1 1]
 [1 1 1]
 [1 0 0]
 [1 0 0]
 [1 0 1]
 [1 0 1]]
Bootstrapped y for tree 0:
 [1 1 0 1 1 1 0 0 1 1]

----------------------------------------------------------------------------------------------------

Bootstrapped X for tree 1:
 [[1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 1 1]
 [1 1 1]
 [0 0 0]
 [1 0 1]
 [1 1 1]
 [1 0 0]]
Bootstrapped y for tree 1:
 [0 0 1 0 1 1 0 1 1 0]

----------------------------------------------------------------------------------------------------

Bootstrapped X for tree 2:
 [[1 0 0]
 [1 1 1]
 [0 1 0]
 [1 1 1]
 [0 0 0]
 [1 0 0]
 [1 0 1]
 [1 0 1]
 [1 1 1]
 [0 0 0]]
Bootstrapped y for tree 2:
 [0 1 1 1 0 0 1 1 1 0]


In [64]:
# Generate a tree for each bootstrapped sample
trees = []
for i in range(B):
    tree = []
    root_indices = range(len(bootstrapped_X[i]))
    build_tree_recursive(bootstrapped_X[i], bootstrapped_y[i], root_indices, "Root", max_depth=2, current_depth=0)
    trees.append(tree)
    print()
    

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 4, 5, 8, 9]
  -- Right leaf node with indices []
- Depth 1, Right: Split on feature: 0
  -- Left leaf node with indices [2, 6, 7]
  -- Right leaf node with indices [1, 3]

 Depth 0, Root: Split on feature: 1
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [4, 5, 8]
  -- Right leaf node with indices [2]
- Depth 1, Right: Split on feature: 2
  -- Left leaf node with indices [7]
  -- Right leaf node with indices [0, 1, 3, 6, 9]

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [1, 3, 6, 7, 8]
  -- Right leaf node with indices []
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [2]
  -- Right leaf node with indices [0, 4, 5, 9]



In [66]:
# Function to predict a new sample
def predict(sample, tree):
    """
    Predicts the label for a single sample by 
    recursively descending the tree until a leaf node is hit
    
    Args:
        sample (ndarray): Data matrix of shape(n_features)
        tree (list):      List of tuples containing the tree information
    
    Returns:
        label (int):      Label predicted for the sample
    """
    
    # You need to return the following variables correctly
    label = None
    
    # Descend the tree until a leaf node is hit
    for node in tree:
        left_indices, right_indices, feature = node
        
        if sample[feature] == 1:
            if len(left_indices) == 0:
                label = 0
                break
            else:
                node = left_indices
        else:
            if len(right_indices) == 0:
                label = 1
                break
            else:
                node = right_indices
    
    return label

In [41]:
# Majority vote
def majority_vote(predictions):
    """
    Returns the majority label from a list of predictions
    
    Args:
        predictions (list): List of predictions
    
    Returns:
        label (int):        Majority label
    """
    
    # You need to return the following variables correctly
    label = None
    
    for i in range(len(predictions)):
        if predictions[i] == None:
            predictions[i] = 0
            
    p1 = len(predictions[predictions==1])
    p0 = len(predictions[predictions==0])
    
    if p1 > p0:
        label = 1
    else:
        label = 0
    
    return label

# XGBoost

A continuación resolveremos el mismo ejemplo con XGBoost, para comparar los resultados.

### 6.1  Importar paquetes

Será necesario haber instalado librerías adicionales:

- `pip install xgboost`
- `pip install scikit-learn`

In [67]:
# Importar paquetes para XGBoost
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


Vamos a dividir los datos en dos subconjuntos: entrenamiento y test.

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [77]:
print(X_train)
print(X_test)


[[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]
 [0 1 1]
 [0 0 0]
 [1 0 1]
 [0 1 0]
 [1 0 0]]
[[0 1 0]]


Transformamos los datos a formato `DMatrix` de XGBoost:

In [78]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

Establecemos los parámetros de trabajo:
 - Profundidad máxima del árbol: `max_depth`
 - Tasa de aprendizaje: `eta`
 - Objetivo: `objective`
 - Número de árboles: `num_round`

In [79]:
param = {
    'max_depth': 2,
    'eta': 1,
    'objective': 'binary:logistic'
}
num_round = 10

Entrenamos/construimos el árbol:

In [80]:
bst = xgb.train(param, dtrain, num_round)

Y lo probamos con el ejemplo que hemos separado para test:

In [81]:
# Predictions
preds = bst.predict(dtest)
predictions = [round(value) for value in preds]

Para por último evaluar la precisión del modelo:

In [83]:
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")
print()
for i, x in enumerate(X_test):
    print(f"X: {x} - y: [{y_test[i]}] --> y_pred: {predictions[i]}")

Accuracy: 0.00%

X: [0 1 0] - y: [1] --> y_pred: 0
